# Imports

In [1]:
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td

import gc
import typing as tp
import wandb
import warnings
import json
import optuna

from pathlib import Path
from itertools import product
from tqdm.autonotebook import tqdm, trange
from joblib import Parallel, delayed
from pprint import pprint

warnings.filterwarnings("ignore", category=DeprecationWarning)
torch.manual_seed(42)

In [2]:
from typing import Callable, Literal
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self,
                 patience: int = 7,
                 threshold = 0,
                 threshold_mode: Literal['rel', 'abs'] = 'abs',
                 verbose: bool = False,
                 path: str = 'checkpoint.pt',
                 trace_func: Callable = print
                 ):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.threshold_mode = threshold_mode
        self.counter = 0
        self.best_val_loss = None
        self.early_stop = False
        self.val_loss_min = torch.inf
        self.threshold = threshold
        self.path = path
        self.trace_func = trace_func

    def _significant_improvement(self, val_loss) -> bool:
        if self.threshold_mode == 'abs':
            return self.best_val_loss - val_loss > self.threshold
        else:
            return (self.best_val_loss - val_loss) / self.best_val_loss > self.threshold

    def __call__(self, val_loss, model):
        # Check if validation loss is nan
        if torch.isnan(val_loss):
            self.trace_func("Validation loss is NaN. Ignoring this epoch.")
            return

        if self.best_val_loss is None:
            self.best_val_loss = val_loss
            # self.save_checkpoint(val_loss, model)
        elif self._significant_improvement(val_loss):
            self.best_val_loss = val_loss
            # self.save_checkpoint(val_loss, model)
            self.counter = 0  # Reset counter since improvement occurred
        else:
            # No significant improvement
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss: float, model: torch.nn.Module):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Load data

In [3]:
# folder = Path("/content/")
folder = Path("data/data_kion")
users_df = pd.read_parquet(folder / "users_final.parquet")
items_df = pd.read_parquet(folder / "items_final.parquet")
train_data = pd.read_parquet(folder / "train_triplets.parquet")
val_data   = pd.read_parquet(folder / "val_pos.parquet")

items_df.sort_index(inplace=True)  # Ensure items are sorted by their IDs

In [4]:
NO_MOVIE = int(items_df.index.max()) + 1 # id bigger than any item id
PAD_SIZE = 30  # users' interactions are padded to have the same number of items
MOVIE_IDS_DTYPE = torch.int16
USER_IDS_DTYPE = torch.int32
MAIN_DTYPE = torch.float32

ITEMS_NUM_GENRE_FEATURES = 29
ITEMS_NUM_CAT_FEATURES = 4
NO_MOVIE

15963

# Dataset

In [5]:
def pad_with_specific_value(tensor, target_length, pad_value):
    """`tensor` is considered to have only unique values"""
    shuffled = tensor[torch.randperm(len(tensor))[:target_length]]
    cur_length = len(shuffled)
    if cur_length < target_length:
        padding = torch.full((target_length - cur_length,), pad_value, dtype=shuffled.dtype)
        return torch.cat([shuffled, padding])
    else:
        return shuffled

def group_by_user(triplets: pd.DataFrame, column: Literal['film_pos', 'film_neg'] = 'film_pos') -> pd.DataFrame:
    """Group interactions by user and convert to DataFrame with column 'interactions' - Tensors of unique film IDs"""
    groupped_users = triplets.groupby('user_id').apply(lambda x: x[column].tolist())
    groupped_users = pd.DataFrame({'interactions': groupped_users.values}, index=groupped_users.index)
    groupped_users['interactions'] = groupped_users['interactions'].apply(lambda x: torch.unique(torch.tensor(x, dtype=MOVIE_IDS_DTYPE)))
    return groupped_users

In [6]:
def collate_fn(data: list[tuple]):
    return data

class BaseDSSMDataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.type = None
        self.seen_items = {}

class TrainDSSMDataset(BaseDSSMDataset):
    def __init__(self, triplets: pd.DataFrame):
        super().__init__()
        self.type = 'train'
        self.triplets = triplets
        self.grouped_pos_users_interactions = group_by_user(triplets, column='film_pos')  # just like padded users but not padded
        self.padded_users = self.grouped_pos_users_interactions.copy(deep=True)
        self.padded_users['interactions'] = self.padded_users['interactions'].apply(lambda x: pad_with_specific_value(x, PAD_SIZE, NO_MOVIE).to(dtype=torch.int32))  # int32 required by EmbeddingBag

        self.grouped_neg_users_interactions = group_by_user(triplets, column='film_neg')  # all negative interactions grouped by users into tensors
        self.all_users = torch.tensor(self.grouped_pos_users_interactions.index.tolist(), dtype=USER_IDS_DTYPE)  # all unique users from triplets

    def __getitem__(self, index: int):
        cur_triplet = self.triplets.iloc[index]
        user_id = cur_triplet['user_id']

        user_info = torch.tensor(users_df.loc[user_id].values, dtype=MAIN_DTYPE)
        user_interactions = self.padded_users.loc[user_id]['interactions']
        pos_films_features = torch.tensor(items_df.loc[cur_triplet['film_pos']].values, dtype=MAIN_DTYPE)
        neg_films_features = torch.tensor(items_df.loc[cur_triplet['film_neg']].values, dtype=MAIN_DTYPE)
        pos_ids = self.grouped_pos_users_interactions.loc[user_id]['interactions']

        return user_id, user_interactions, user_info, pos_films_features, neg_films_features, pos_ids

    def __getitems__(self, index: tp.Sequence[int]):
        cur_triplets = self.triplets.iloc[index]
        user_ids = cur_triplets['user_id']

        user_info = torch.tensor(users_df.loc[user_ids].values, dtype=MAIN_DTYPE)
        user_interactions = torch.stack(self.padded_users.loc[user_ids]['interactions'].tolist())
        pos_films_features = torch.tensor(items_df.loc[cur_triplets['film_pos']].values, dtype=MAIN_DTYPE)
        neg_films_features = torch.tensor(items_df.loc[cur_triplets['film_neg']].values, dtype=MAIN_DTYPE)
        pos_ids = self.grouped_pos_users_interactions.loc[user_ids]['interactions'].tolist()

        return user_ids, user_interactions, user_info, pos_films_features, neg_films_features, pos_ids

    def get_user_data(self, user_id: int) -> tuple[Tensor, Tensor]:
        user_info = torch.tensor(users_df.loc[user_id].values, dtype=MAIN_DTYPE)
        if user_id in self.padded_users.index:
            user_interactions = self.padded_users.loc[user_id]['interactions']
        else:
            user_interactions = torch.full((PAD_SIZE,), NO_MOVIE, dtype=torch.int32)

        return user_interactions, user_info

    def get_users_data(self, user_ids: int) -> tuple[Tensor, Tensor]:
        user_info = torch.tensor(users_df.loc[user_ids].values, dtype=MAIN_DTYPE)

        inters = []
        for uid in user_ids:
            if uid in self.padded_users.index:
                inter = self.padded_users.loc[uid, 'interactions']
            else:
                inter = torch.full(
                    (PAD_SIZE,),
                    NO_MOVIE,
                    dtype=torch.int32
                )
            inters.append(inter)

        user_interactions = torch.stack(inters, dim=0)

        return user_interactions, user_info

    def __len__(self):
        return len(self.triplets)

class EvalDSSMDataset(BaseDSSMDataset):
    def __init__(self, pos_df: pd.DataFrame):
        super().__init__()
        self.type = 'eval'
        self.all_users = pos_df['user_id'].unique().astype(int)
        self.grouped_pos_users_interactions = group_by_user(pos_df, column='film_pos')
        # self.seen_items = train_triplets.grouped_pos_users_interactions | train_triplets.grouped_neg_users_interactions  # seen items are all positive and negative interactions from train triplets

    def __getitem__(self, index: int):
        user = int(self.all_users[index])
        pos  = int(self.grouped_pos_users_interactions[user]['interactions'])
        return user, pos

    def __getitems__(self, index: tp.Sequence[int]):
        users = self.all_users[index]
        pos   = self.grouped_pos_users_interactions.loc[users]['interactions'].tolist()
        return users, pos

    def get_all_users(self) -> Tensor:
        """
        Get IDs of all users, that have positive interactions in this dataset

        Returns:
            users (Tensor, dtype=torch.int32) : All user IDs
        """
        return self.all_users

    def __len__(self):
        return len(self.all_users)


# Model

In [7]:
class ItemNet(nn.Module):
    def __init__(self,
                 dim_embedding: int,
                 dim_input: int,
                 dim_hidden: int,
                 activation: tp.Callable[[Tensor], Tensor] = nn.ReLU()
                 ) -> None:
        super().__init__()
        self.cat_embedding = nn.Linear(ITEMS_NUM_CAT_FEATURES, dim_embedding)
        self.genre_embedding = nn.Linear(ITEMS_NUM_GENRE_FEATURES, dim_embedding)
        self.country_embedding = nn.Linear(dim_input - ITEMS_NUM_CAT_FEATURES - ITEMS_NUM_GENRE_FEATURES, dim_embedding)
        self.dense_block = nn.Sequential(
            nn.Linear(dim_embedding * 3, dim_hidden),
            activation,
        )
        self.output_layer = nn.Linear(dim_hidden, dim_embedding, bias=False)
        self.norm = nn.LayerNorm(dim_embedding)

    def forward(self, item_features: Tensor) -> Tensor:
        cat_features = item_features[:, :ITEMS_NUM_CAT_FEATURES]
        genre_features = item_features[:, ITEMS_NUM_CAT_FEATURES:ITEMS_NUM_CAT_FEATURES + ITEMS_NUM_GENRE_FEATURES]
        country_features = item_features[:, ITEMS_NUM_CAT_FEATURES + ITEMS_NUM_GENRE_FEATURES:]

        cat_emb = self.cat_embedding(cat_features)
        genre_emb = self.genre_embedding(genre_features)
        country_emb = self.country_embedding(country_features)

        all_features_emb = torch.concat([cat_emb, genre_emb, country_emb], axis=1)
        features = self.dense_block(all_features_emb)
        output = self.output_layer(features)

        return self.norm(output)


class UserNet(nn.Module):
    def __init__(self,
                 dim_embedding: int,
                 num_items: int,
                 dim_input: int,
                 dim_hidden: int,
                 activation: tp.Callable[[Tensor], Tensor] = nn.ReLU()
                 ) -> None:                              # | +1 for the NO_MOVIE element
        super().__init__()                               # V
        self.interacted_items_embeddings = nn.EmbeddingBag(num_items + 1,
                                                           dim_embedding,
                                                           mode="mean",
                                                           padding_idx=num_items)
        self.info_embedding = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_embedding),
            activation
        )
        self.dense_layer = nn.Sequential(
            nn.Linear(dim_embedding, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_embedding),
            activation
        )
        self.output_layer = nn.Linear(dim_embedding * 3, dim_embedding, bias=False)
        self.norm = nn.LayerNorm(dim_embedding)
        self.num_items = num_items
        self.dim_user_features = dim_input

    def forward(self, user_interactions: Tensor, user_info: Tensor) -> Tensor:
        interactions_emb = self.interacted_items_embeddings(user_interactions)
        info_emb = self.info_embedding(user_info.float())
        features = self.dense_layer(interactions_emb)
        x = torch.cat([interactions_emb, features, info_emb], dim=1)
        output = self.output_layer(x)
        return self.norm(output)

In [8]:
class DSSM(nn.Module):
    def __init__(self,
                 dim_item_features: int,
                 dim_user_features: int,
                 num_items: int,
                 dim_embedding: int = 2048,
                 dim_hidden: int = 1024, 
                 ) -> None:
        super().__init__()
        self.item_net = ItemNet(dim_embedding=dim_embedding,
                                dim_input=dim_item_features,
                                dim_hidden=dim_hidden)
        self.user_net = UserNet(dim_embedding=dim_embedding,
                                num_items=num_items,
                                dim_input=dim_user_features,
                                dim_hidden=dim_hidden)

    def forward(self,
                user_intercations: Tensor,
                user_info: Tensor,
                item_features_pos: Tensor,
                item_features_neg: Tensor,
                ) -> tuple[Tensor, Tensor, Tensor]:
        """Returns embeddings of users and embeddings of their positive and negative items"""
        anchor = self.user_net(user_intercations, user_info)
        pos = self.item_net(item_features_pos)
        neg = self.item_net(item_features_neg)

        return anchor, pos, neg

    def encode_user(self, user_interactions: Tensor, user_info: Tensor) -> Tensor:
        return self.user_net(user_interactions, user_info)

    def encode_item(self, item_features: Tensor) -> Tensor:
        return self.item_net(item_features)

# Training

## Metrics

In [9]:
# def precision_at_k(y_true_sets, y_pred, k):
#     precisions = []
#     for true_set, pred in zip(y_true_sets, y_pred):
#         pred_k = pred[:k]
#         precisions.append(len(set(pred_k) & true_set) / k)
#     return np.mean(precisions)

# def _dcg(rel, k):
#     score = 0.0
#     for i, r in enumerate(rel[:k]):
#         score += r / log2(i + 2)
#     return score

# def _ndcg_single(true_set, pred, k):
#     rel = [1 if item in true_set else 0 for item in pred[:k]]
#     dcg = _dcg(rel, k)
#     ideal = sorted(rel, reverse=True)
#     idcg = _dcg(ideal, k)
#     return dcg / idcg if idcg > 0 else 0.0

# def ndcg_at_k(y_true_sets, y_pred, k, n_jobs=-1):
#     ndcgs = Parallel(n_jobs=n_jobs)(
#         delayed(_ndcg_single)(ts, pred, k)
#         for ts, pred in tqdm(zip(y_true_sets, y_pred), desc='Calculating NDCG@k', total=len(y_true_sets))
#     )
#     return np.mean(ndcgs)

# def _mrr_single(true_set, pred, k):
#     for rank, item in enumerate(pred[:k], start=1):
#         if item in true_set:
#             return 1.0 / rank
#     return 0.0

# def mrr_at_k(y_true_sets, y_pred, k, n_jobs=-1):
#     mrrs = Parallel(n_jobs=n_jobs)(
#         delayed(_mrr_single)(ts, pred, k)
#         for ts, pred in tqdm(zip(y_true_sets, y_pred), desc='Calculating MRR@k', total=len(y_true_sets))
#     )
#     return np.mean(mrrs)

def _recall_single(relevant_tensor, pred_tensor, k):
    if (num_relevant_items := len(relevant_tensor)) == 0:
        return None
    return torch.isin(pred_tensor[:k], relevant_tensor).sum().float() / num_relevant_items

def recall_at_k(y_true, y_pred, k, n_jobs=-1):
    recalls = Parallel(n_jobs=n_jobs)(
        delayed(_recall_single)(ts, pred, k)
        for ts, pred in tqdm(zip(y_true, y_pred),
                             desc=f'Calculating Recall@{k}',
                             total=len(y_true),
                             leave=False)
    )
    return sum(recalls) / len(recalls)

def _average_precision_single_full(relevant_ids: Tensor, predicted_ids: Tensor, k: int = None):
    """
    Calculate mean Average Precision (mAP) for recommendation systems.

    Args:
        relevant_ids: Tensor of relevant item IDs (ground truth)
        predicted_ids: Tensor of predicted item IDs sorted by relevance
        k: Optional cutoff for top-k items to consider. If None, use all predicted items.

    Returns:
        mAP score
    """
    if (num_relevant := len(relevant_ids)) == 0:
        return torch.nan

    if k is not None:
        predicted_ids = predicted_ids[:k]

    # Create binary relevance tensor (1 for relevant items, 0 otherwise)
    relevance = torch.isin(predicted_ids, relevant_ids).float()

    # Calculate precision at each position
    cum_relevance = torch.cumsum(relevance, dim=0)
    positions = torch.arange(1, len(predicted_ids) + 1)
    precision_at_k = cum_relevance / positions

    # Only consider precision at positions where item is relevant
    relevant_precision = precision_at_k * relevance

    # Sum relevant precisions and divide by total relevant items
    ap = torch.sum(relevant_precision) / num_relevant

    return ap


def mean_average_precision_full(y_true: list[Tensor],
                                y_pred: list[Tensor],
                                k: int | None = None,
                                n_jobs: int = -1,
                                ) -> float:
    """
    Compute MAP over full prediction list.
    """
    ap_scores = Parallel(n_jobs=n_jobs)(
        delayed(_average_precision_single_full)(true, pred, k)
        for true, pred in tqdm(zip(y_true, y_pred),
                               desc='Calculating MAP (no K)',
                               total=len(y_true),
                               leave=False)
    )
    return torch.nanmean(torch.tensor(ap_scores, dtype=torch.float)).item()

## Configure training

In [10]:
print('Get train dataset')
train_dataset = TrainDSSMDataset(train_data)
print('Get validation dataset')
val_dataset = EvalDSSMDataset(val_data)

Get train dataset
Get validation dataset


In [11]:
# Hyperparams
EPOCHS = 1
BATCH_SIZE = 1024 * 16
NUM_WORKERS = 0
METRICS_N_JOBS = 12
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

EMBEDDING_DIM = 2048
HIDDEN_DIM = 1024
LR = 1e-2
WEIGHT_DECAY = 1e-5
TRIPLET_LOSS_MARGIN = 0.4
EXPERIMENT_NAME = 'final_testing'
K = 10
LOG_TO_WANDB = False

train_dataloader = td.DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_dataloader = td.DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

model = DSSM(dim_item_features=items_df.shape[1],
             dim_user_features=users_df.shape[1],
             num_items=items_df.shape[0],
             dim_embedding=EMBEDDING_DIM,
             dim_hidden=HIDDEN_DIM,
             ).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, threshold=3e-3, threshold_mode='abs')
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer,
#     patience=3,
#     threshold=1e-3,
#     threshold_mode='abs'
# )
# early_stopping = EarlyStopping(
#     patience=7,
#     threshold=1e-3,
#     threshold_mode='rel',
#     verbose=True,
#     trace_func=tqdm.write,
#     path=f'model_weights/{EXPERIMENT_NAME}.ckpt'
# )

## Training loop

In [12]:
if LOG_TO_WANDB:
    entity = "xenz5240-higher-school-of-economics"
    wandb.init(entity=entity, project='reelsrec-dssm', name=EXPERIMENT_NAME)

    wandb.config.update({
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "embedding_dim": EMBEDDING_DIM,
        "hidden_dim": HIDDEN_DIM,
        "learning_rate": LR,
        "weight_decay": WEIGHT_DECAY,
        "triplet_loss_margin": TRIPLET_LOSS_MARGIN,
        "device": DEVICE
    })

best_map = 0.0

for epoch in trange(EPOCHS, position=0, desc='Training', unit='epoch'):
    model.train()
    epoch_losses = []
    batch_times = []
    for step, batch in enumerate(tqdm(train_dataloader, position=1, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch')):
        _, user_inters, user_info, pos_features, neg_features, _ = batch
        optimizer.zero_grad()
        batch_users_embs, batch_positive_films_embs, batch_negative_films_embs = model(user_inters.to(DEVICE),
                                                                                       user_info.to(DEVICE),
                                                                                       pos_features.to(DEVICE),
                                                                                       neg_features.to(DEVICE))
        loss = F.triplet_margin_loss(batch_users_embs, batch_positive_films_embs, batch_negative_films_embs, margin=TRIPLET_LOSS_MARGIN)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

        if LOG_TO_WANDB and step % 20 == 0:
            wandb.log({"train/loss": loss.item(),
                       "epoch": epoch + 1,
                       "step": step})

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    if LOG_TO_WANDB:
        wandb.log({"train/epoch_loss": mean_train_loss, "epoch": epoch + 1})

    # Evaluation
    model.eval()
    all_predictions = []
    all_references = []
    # Embed all items
    items_df.sort_index(inplace=True)  # Ensure items are sorted by their IDs
    item_features = torch.tensor(items_df.values, dtype=MAIN_DTYPE, device=DEVICE)
    EMPTY_WATCH_HISTORY = torch.empty(0, dtype=MOVIE_IDS_DTYPE)
    with torch.no_grad():
        all_items_embeds = model.encode_item(item_features)

        for step, batch in enumerate(tqdm(val_dataloader, position=1, desc=f'Validation {epoch + 1}/{EPOCHS}', unit='batch')):
            user_ids, positive_interactions = batch
            user_inters, user_info = train_dataset.get_users_data(user_ids)
            user_embs = model.encode_user(user_inters.to(DEVICE), user_info.to(DEVICE))

            # Build distance matrix (Euclidian distance)
            distance_matrix = torch.cdist(user_embs, all_items_embeds, p=2.0)
            val_recommendations = distance_matrix.argsort(dim=1, descending=False).to(dtype=MOVIE_IDS_DTYPE, device='cpu')  # (batch_users x all_items)

            # Filter recommendations by removing watched films
            # Get all films each user has interacted with (from both film_pos and film_neg)
            user_interacted_films: dict[int, Tensor] = (
                train_dataset.triplets[train_dataset.triplets['user_id'].isin(user_ids.tolist())]
                .groupby('user_id')[['film_pos', 'film_neg']]
                .apply(lambda x: torch.tensor(pd.unique(x.values.ravel('K')).tolist(), dtype=MOVIE_IDS_DTYPE))
            ).to_dict()

            # For each user remove watched films from their recommendations
            filtered_val_recommendations = [None] * len(user_ids)
            for i, (user_ids, recommendations) in tqdm(enumerate(zip(user_ids, val_recommendations)),
                                                       desc='Filtering recommendations',
                                                       total=len(user_ids),
                                                       leave=False):
                watched_films = user_interacted_films.get(user_ids, EMPTY_WATCH_HISTORY)
                filtered_val_recommendations[i] = recommendations[torch.isin(recommendations, watched_films, invert=True)]

            all_predictions.extend(filtered_val_recommendations)
            all_references.extend(positive_interactions)
    
    # Calculate metrics
    recall_score = recall_at_k(y_true=all_references, y_pred=all_predictions, k=K, n_jobs=METRICS_N_JOBS)
    map_at_k_score = mean_average_precision_full(y_true=all_references, y_pred=all_predictions, k=K, n_jobs=METRICS_N_JOBS)
    map_score = mean_average_precision_full(y_true=all_references, y_pred=all_predictions, n_jobs=METRICS_N_JOBS)
    tqdm.write(f'Recall@{K}: {recall_score:.5f} | mAP@{K}: {map_at_k_score:.5f} | mAP: {map_score:.5f}')
    # precision = precision_at_k(all_predictions, all_references, k=K)
    # ndcg   = ndcg_at_k(all_predictions, all_references, k=K)
    # mrr    = mrr_at_k(all_predictions, all_references, k=K)
    # tqdm.write(f'Mean val los: {mean_val_loss:.4f} | Recall: {recall:.4f} | Precision: {precision:.4f} | NDCG: {ndcg:.4f} | MRR: {mrr:.4f}')
    torch.cuda.empty_cache()
    gc.collect()
    del all_predictions, all_references, filtered_val_recommendations, positive_interactions, all_items_embeds

    if LOG_TO_WANDB:
        wandb.log({
            f"val/Recall@{K}": recall_score,
            f"val/mAP@{K}": map_at_k_score,
            f"val/mAP": map_score,
            # f"val/precision@{K}": precision,
            # f"val/ndcg@{K}": ndcg,
            # f"val/mrr@{K}": mrr,
            "train/lr": optimizer.param_groups[0]['lr'],
            "epoch": epoch + 1,
        })

Training:   0%|          | 0/1 [00:00<?, ?epoch/s]

Epoch 1/1:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/1:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating Recall@10:   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@10: 0.00046 | mAP@10: 0.00015 | mAP: 0.00112


# Hyperparameters tuning

## Manual grid search

In [ ]:
def train_model(epochs: int = 50, embedding_dim: int = 192, lr: float = 1e-4, weight_decay: float = 1e-5, triplet_loss_margin: float = 0.4, wandb_id: int | str = 0):
    # Hyperparams
    EPOCHS = epochs
    BATCH_SIZE = 1024 * 16
    NUM_WORKERS = 0
    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    EMBEDDING_DIM = embedding_dim
    LR = lr
    WEIGHT_DECAY = weight_decay
    TRIPLET_LOSS_MARGIN = triplet_loss_margin
    EXPERIMENT_NAME = f'improved_validation_{wandb_id}'
    K = 100
    LOG_TO_WANDB = True

    train_dataloader = td.DataLoader(train_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_dataloader = td.DataLoader(val_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model = DSSM(dim_item_features=items_df.shape[1],
                dim_user_features=users_df.shape[1],
                num_items=NO_MOVIE,
                dim_embedding=EMBEDDING_DIM
                ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    # lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, threshold=3e-3, threshold_mode='abs')
    # lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer,
    #     patience=3,
    #     threshold=1e-3,
    #     threshold_mode='abs'
    # )
    # early_stopping = EarlyStopping(
        # patience=7,
        # threshold=1e-3,
        # threshold_mode='rel',
        # verbose=True,
        # trace_func=tqdm.write,
        # path=f'model_weights/{EXPERIMENT_NAME}.ckpt'
    # )
    if LOG_TO_WANDB:
        entity = "xenz5240-higher-school-of-economics"
        wandb.init(entity=entity, project='reelsrec-dssm', name=EXPERIMENT_NAME)

        wandb.config.update({
            "epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
            "embedding_dim": EMBEDDING_DIM,
            "learning_rate": LR,
            "weight_decay": WEIGHT_DECAY,
            "triplet_loss_margin": TRIPLET_LOSS_MARGIN,
            "device": DEVICE
        })
    results = {'mAP': [], 'recall': []}
    for epoch in trange(EPOCHS, position=0, desc='Training', unit='epoch'):
        model.train()
        epoch_losses = []
        for step, batch in enumerate(tqdm(train_dataloader, position=1, desc=f'Epoch {epoch + 1}/{EPOCHS}', unit='batch')):
            _, user_inters, user_info, pos_features, neg_features, _ = batch
            optimizer.zero_grad()
            batch_users_embs, batch_positive_films_embs, batch_negative_films_embs = model(user_inters.to(DEVICE),
                                                                                        user_info.to(DEVICE),
                                                                                        pos_features.to(DEVICE),
                                                                                        neg_features.to(DEVICE))
            loss = F.triplet_margin_loss(batch_users_embs, batch_positive_films_embs, batch_negative_films_embs, margin=TRIPLET_LOSS_MARGIN)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            if LOG_TO_WANDB and step % 20 == 0:
                wandb.log({"train/loss": loss.item(),
                        "epoch": epoch + 1,
                        "step": step})

        mean_train_loss = sum(epoch_losses) / len(epoch_losses)
        if LOG_TO_WANDB:
            wandb.log({"train/epoch_loss": mean_train_loss, "epoch": epoch + 1})

        # Evaluation
        model.eval()
        all_predictions = []
        all_references = []
        # Embed all items
        item_features = torch.tensor(items_df.values, dtype=MAIN_DTYPE, device=DEVICE)
        EMPTY_WATCH_HISTORY = torch.empty(0, dtype=MOVIE_IDS_DTYPE)
        with torch.no_grad():
            all_items_embeds = model.encode_item(item_features)

            for step, batch in enumerate(tqdm(val_dataloader, position=1, desc=f'Validation {epoch + 1}/{EPOCHS}', unit='batch')):
                user_ids, positive_interactions = batch
                user_inters, user_info = train_dataset.get_users_data(user_ids)
                user_embs = model.encode_user(user_inters.to(DEVICE), user_info.to(DEVICE))

                # строим матрицу расстояний
                distance_matrix = torch.cdist(user_embs, all_items_embeds, p=2.0)
                val_recommendations = distance_matrix.argsort(dim=1, descending=False).to(dtype=MOVIE_IDS_DTYPE, device='cpu')  # (batch_users x all_items)

                # фильтруем рекомендации, убирая просмотренные фильмы
                # Get all films each user has interacted with (from both film_pos and film_neg)
                user_interacted_films: dict[int, Tensor] = (train_dataset.triplets[train_dataset.triplets['user_id'].isin(user_ids.tolist())]
                                                                                   .groupby('user_id')[['film_pos', 'film_neg']]
                                                                                   .apply(lambda x: torch.tensor(pd.unique(x.values.ravel('K')).tolist(), dtype=MOVIE_IDS_DTYPE))
                                                            ).to_dict()

                # For each user remove watched films from their recommendations
                filtered_val_recommendations = [None] * len(user_ids)
                for i, (user_ids, recommendations) in tqdm(enumerate(zip(user_ids, val_recommendations)),
                                                           desc='Filtering recommendations',
                                                           total=len(user_ids),
                                                           leave=False):
                    watched_films = user_interacted_films.get(user_ids, EMPTY_WATCH_HISTORY)
                    filtered_val_recommendations[i] = recommendations[torch.isin(recommendations, watched_films, invert=True)]

                all_predictions.extend(filtered_val_recommendations)
                all_references.extend(positive_interactions)

        map_score = mean_average_precision_full(y_true=all_references, y_pred=all_predictions, n_jobs=8)
        results['mAP'].append(map_score)
        recall_score = float(recall_at_k(y_true=all_references, y_pred=all_predictions, k=K, n_jobs=8))
        results['recall'].append(recall_score)
        tqdm.write(f'Recall@{K}: {recall_score:.5f} | mAP: {map_score:.5f}')

        del all_predictions, all_references, filtered_val_recommendations, positive_interactions, all_items_embeds
        gc.collect()

        if LOG_TO_WANDB:
            wandb.log({
                f"val/recall@{K}": recall_score,
                f"val/mAP": map_score,
                "train/lr": optimizer.param_groups[0]['lr'],
                "epoch": epoch + 1,
            })
    results['Average mAP'] = sum(results['mAP']) / len(results['mAP'])
    results['Max mAP'] = max(results['mAP'])
    results['Average recall'] = sum(results['recall']) / len(results['recall'])
    results['Max recall'] = max(results['recall'])
    results['params'] = {'embedding_dim': EMBEDDING_DIM,
                         'lr': LR,
                         'weight_decay': WEIGHT_DECAY,
                         'triplet_loss_margin': TRIPLET_LOSS_MARGIN}
    del results['mAP'], results['recall']
    return results

In [ ]:
params_grid = {'epochs': [25],
               'embedding_dim': [512, 768, 1024],
               'lr': [5e-3, 1e-3],
               'weight_decay': [1e-5],
               'triplet_loss_margin': [0.4],
               'wandb_id': [11]}
grid_results = []
for params_set in [dict(zip(params_grid.keys(), values)) for values in product(*params_grid.values())]:
    # pprint(params_set)
    cur_res = train_model(**params_set)
    grid_results.append(cur_res)

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇█
step,▁▃▄▅▅▇▄▆▇█▄▇▄▂▄▆▂▅▅▅▅▇▁▄▂▄▅▆▇▄▂▃▇▁▆▁█▅▂▅
train/epoch_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP,▁▂▂▅▇▅▇▄▃▃▇▅▃▅▄▃▃█▂█▅▅▄▄▄▅▃▃▄▃▃▃▃▂▄▆▄▅▄▂
val/recall@100,▁▄▃▇▇▆▇██▇▇▇▇▆▇█▇▇█▄▇█▇▆▇██▇▇▇▄▆▅▆▇█▇▇▇▆
epoch,50
step,220
train/epoch_loss,0.01002
train/loss,0.01076


Training:   0%|          | 0/25 [00:00<?, ?epoch/s]

Epoch 1/25:   0%|          | 0/235 [00:00<?, ?batch/s]

Validation 1/25:   0%|          | 0/3 [00:00<?, ?batch/s]

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

...

Filtering recommendations:   0%|          | 0/16384 [00:00<?, ?it/s]

Filtering recommendations:   0%|          | 0/13097 [00:00<?, ?it/s]

Calculating MAP (no K):   0%|          | 0/45865 [00:00<?, ?it/s]

Calculating Recall@100:   0%|          | 0/45865 [00:00<?, ?it/s]

Recall@100: 0.07404 | mAP: 0.02143


## Optuna study
Maximizing average and max mAP values accross all validations

In [14]:
def objective(trial: optuna.Trial):
    params = {
        'embedding_dim': trial.suggest_int('embedding_dim', 192, 2048),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'triplet_loss_margin': trial.suggest_float('triplet_loss_margin', 0.01, 5)
    }
    print('Trying params:')
    pprint(params)
    cur_res = train_model(epochs=25, wandb_id='optuna', **params)
    cur_res['params'] = params
    with open('study_results.json', 'r') as f:
        previous_runs: list[dict] = json.load(f)
    previous_runs.append(cur_res)
    with open('study_results.json', 'w') as f:
        json.dump(sorted(previous_runs, key=lambda r: -r['Max mAP']), f)

    torch.cuda.empty_cache()
    gc.collect()
    return cur_res['Max mAP'], cur_res['Average mAP']

# Create a study
study = optuna.create_study(
    directions=['maximize', 'maximize'],
    study_name='dssm_optimization_2',
    storage='sqlite:///optuna_study.db',  # Optional: for persistence
    load_if_exists=True
)

# with open('study_results.json', 'r') as f:
#     previous_runs = json.load(f)

# # Add the previous runs to the study
# for run in previous_runs:
#     if 'wandb_id' in run['params']:
#         del run['params']['wandb_id']
#     trial = optuna.trial.create_trial(
#         params=run['params'],
#         distributions={
#             'embedding_dim': optuna.distributions.IntDistribution(192, 2048),
#             'lr': optuna.distributions.FloatDistribution(1e-4, 1e-2, log=True),
#             'weight_decay': optuna.distributions.FloatDistribution(1e-5, 1e-3, log=True),
#             'triplet_loss_margin': optuna.distributions.FloatDistribution(0.01, 10)
#         },
#         values=(run['Max mAP'], run['Average mAP'])
#     )
#     study.add_trial(trial)

# Start optimization
study.optimize(func=objective,
               n_trials=50,
               gc_after_trial=True)

# Print best results
print(f"Best trial:")
trial = study.best_trial
print(f"  Metrics: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-06-08 00:26:17,398] Using an existing study with name 'dssm_optimization_2' instead of creating a new one.


Trying params:
{'embedding_dim': 1438,
 'lr': 0.0002311154780075968,
 'triplet_loss_margin': 0.056615340771104264,
 'weight_decay': 1.4023654511062853e-05}


wandb: Currently logged in as: sergfedchin (xenz5240-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/25: 100%|██████████| 235/235 [03:11<00:00,  1.22batch/s]
















































































































































































































Training:   0%|          | 0/25 [04:28<?, ?epoch/s]

Recall@100: 0.00732 | mAP: 0.00091


...

Recall@100: 0.04847 | mAP: 0.00424


Epoch 14/25: 100%|██████████| 235/235 [00:36<00:00,  6.48batch/s]

























































































































































































Training:  52%|█████▏    | 13/25 [22:21<19:09, 95.80s/epoch]

Recall@100: 0.04924 | mAP: 0.00473


Epoch 15/25: 100%|██████████| 235/235 [00:34<00:00,  6.82batch/s]


































































Training:  56%|█████▌    | 14/25 [23:16<18:17, 99.77s/epoch]
[W 2025-06-08 19:15:19,716] Trial 38 failed with parameters: {'embedding_dim': 381, 'lr': 0.00010045612439363125, 'weight_decay': 1.1366112710498758e-05, 'triplet_loss_margin': 2.3916476674323652} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/serg_fedchn/Homework/6_semester/DL/reelrecs/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_556697/3506072288.py", line 10, in objective
    cur_res = train_model(epochs=25, wandb_id='optuna', **params)
  File "/tmp/ipykernel_556697/3978670567.py", line 98, in train_model
    val_recommendations = distance_matrix.argsort(dim=1, descending=False).to(dtype=MOVIE_IDS_DTYPE, device='cpu')  # (batch_users x all_it

KeyboardInterrupt: 

In [18]:
with open('study_results.json', 'r') as f:
    previous_runs: list[dict] = json.load(f)
pprint(sorted(previous_runs, key=lambda r: r['Max mAP']))

[{'Average mAP': 0.0006300935812769312,
  'Average recall': 0.0029991762712597847,
  'Max mAP': 0.0008679295550692672,
  'Max recall': 0.005420859903097153,
  'params': {'embedding_dim': 192,
             'lr': 0.0001,
             'triplet_loss_margin': 0.01,
             'wandb_id': 4,
             'weight_decay': 1e-05}},
 {'Average mAP': 0.0007481924624445408,
  'Average recall': 0.005650196485221386,
  'Max mAP': 0.0009063473342442375,
  'Max recall': 0.0073239305056631565,
  'params': {'embedding_dim': 1438,
             'lr': 0.0002311154780075968,
             'triplet_loss_margin': 0.056615340771104264,
             'weight_decay': 1.4023654511062853e-05}},
 {'Average mAP': 0.0013414419810199453,
  'Average recall': 0.012023180648684502,
  'Max mAP': 0.001848413588716142,
  'Max recall': 0.017583943903446198,
  'params': {'embedding_dim': 1416,
             'lr': 0.0004892795861761351,
             'triplet_loss_margin': 0.6414599959241384,
             'weight_decay': 0.00084

## Final tries

In [ ]:
for dim_embedding in [1024, 1536, 2048]:
    for lr in [1e-3, 5e-3, 1e-2]:
        if lr in [1e-3, 5e-3] and dim_embedding == 1024:
            continue
        params = {
            'embedding_dim': dim_embedding,
            'lr': lr,
            'weight_decay': 1e-5,
            'triplet_loss_margin': 0.4
        }
        print('Trying params:')
        pprint(params)
        cur_res = train_model(epochs=25, wandb_id='final_tries', **params)
        cur_res['params'] = params
        with open('study_results.json', 'r') as f:
            previous_runs: list[dict] = json.load(f)
        previous_runs.append(cur_res)
        with open('study_results.json', 'w') as f:
            json.dump(sorted(previous_runs, key=lambda r: -r['Max mAP']), f)
        
        torch.cuda.empty_cache()
        gc.collect()

Trying params:
{'embedding_dim': 1024,
 'lr': 0.01,
 'triplet_loss_margin': 0.4,
 'weight_decay': 1e-05}


wandb: Currently logged in as: sergfedchin (xenz5240-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/25: 100%|██████████| 235/235 [01:44<00:00,  2.25batch/s]














































































































































































































Training:   0%|          | 0/25 [03:00<?, ?epoch/s]

Recall@100: 0.02183 | mAP: 0.00198


Epoch 2/25: 100%|██████████| 235/235 [01:51<00:00,  2.11batch/s]






























































































































































































Training:   4%|▍         | 1/25 [05:59<1:12:23, 180.97s/epoch]

...

Epoch 11/25: 100%|██████████| 235/235 [06:37<00:00,  1.69s/batch]




















































































































































































Training:  40%|████      | 10/25 [1:26:33<1:58:57, 475.84s/epoch]

Recall@100: 0.07840 | mAP: 0.03229


Epoch 12/25: 100%|██████████| 235/235 [06:37<00:00,  1.69s/batch]















































































































































































Training:  44%|████▍     | 11/25 [1:34:26<1:50:56, 475.47s/epoch]

Recall@100: 0.05716 | mAP: 0.01693


Training:  48%|████▊     | 12/25 [1:34:26<1:42:52, 474.80s/epoch]